In [1]:
%cd ../src

/home/ubuntu/SPVD_Lightning/src


In [2]:
from models.ddpm_unet_cattn import SPVUnet
import torch
import lightning as L
from models.g_spvd import GSPVD

/opt/conda/envs/spvd/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [3]:
model_args = {
    'voxel_size' : 0.1,
    'nfs' : (32, 64, 128, 256), 
    'attn_chans' : 8, 
    'attn_start' : 3, 
    'cross_attn_chans' : 8, 
    'cross_attn_start' : 2, 
    'cross_attn_cond_dim' : 768,
}

model = SPVUnet(**model_args)
model = GSPVD(model=model, lr=0.1, training_steps=1)

In [4]:
from dataloaders.shapenet.shapenet_loader import ShapeNet

categories = ['chair']
path = "../data/ShapeNet"

te = ShapeNet(path, "val", 2048, categories, load_renders=True)

Loading (val) renders for chair (03001627): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 662/662 [00:17<00:00, 37.64it/s]


In [5]:
import numpy as np

samples = 16
references = [te[idx] for idx in np.random.choice(list(range(len(te))), size=(samples,))]

reference_images = torch.stack([r["render-features"] for r in references]).to("cuda")

In [16]:
steps = 125
scheduler = 'ddpm'
ckpt_path = f'../checkpoints/distillation/GSPVD/{"-".join(categories)}/{steps}-steps.ckpt'
ckpt = torch.load(ckpt_path, weights_only=True)
# ckpt_path = '../checkpoints/distillation/GSPVD/starting.ckpt'
# ckpt = torch.load(ckpt_path, weights_only=True)['state_dict']
model.load_state_dict(ckpt)

<All keys matched successfully>

In [17]:
model = model.cuda().eval()

In [18]:
from my_schedulers.ddpm_scheduler import DDPMSparseScheduler
from my_schedulers.ddim_scheduler import DDIMSparseScheduler
from utils.visualization import display_pointclouds_grid

if scheduler == 'ddim':
    sched = DDIMSparseScheduler(beta_min=0.0001, beta_max=0.02, steps=steps, init_steps=1000)
else:
    sched = DDPMSparseScheduler(beta_min=0.0001, beta_max=0.02, steps=steps, init_steps=1000)

In [ ]:
preds = sched.sample(model, samples, 2048, reference=reference_images)

Sampling:  85%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍                        | 106/125 [00:05<00:00, 20.81it/s]

In [ ]:
display_pointclouds_grid(preds.cpu().numpy(), offset=8, point_size=0.3)

In [15]:
real = torch.stack([r["pc"] for r in references]).numpy()
display_pointclouds_grid(real, offset=8, point_size=0.3)

Output()